<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/computer-vision/4_Pretrained_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory

In [2]:
from google.colab import files
r=files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c dogs-vs-cats

100% 812M/812M [00:05<00:00, 148MB/s]
100% 812M/812M [00:05<00:00, 159MB/s]


In [5]:
!unzip -qq dogs-vs-cats.zip

In [6]:
!unzip -qq train.zip

In [7]:
import os, pathlib, shutil

In [8]:
src_dir = pathlib.Path("train")
dst_dir = pathlib.Path("cats-vs-dogs")

In [9]:
def create_directory(name, start_index, end_index):

  for category in ["cat", "dog"]:

    sub_dir = dst_dir / name/ category
    os.makedirs(sub_dir)

    for i in range(start_index, end_index):
      src_file = src_dir / f"{category}.{i}.jpg"
      shutil.copy(src=src_file, dst=sub_dir)

In [10]:
create_directory("train", 0, 1000)
create_directory("validation", 1000, 1500)
create_directory("test", 1500, 2500)

In [11]:
train_dataset = image_dataset_from_directory(dst_dir / "train", image_size = (180, 180), batch_size=32)
validation_dataset = image_dataset_from_directory(dst_dir / "validation", image_size = (180, 180), batch_size=32)
test_dataset = image_dataset_from_directory(dst_dir / "test", image_size = (180, 180), batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [12]:
base_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(180, 180, 3))
base_model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                              

In [13]:
base_model_with_top_layer = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_shape=(224, 224, 3))
base_model_with_top_layer.summary()

553476096/553467096 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

## Extract base model (VGG16) features and labels and add classification layers to train and validate. 

In [14]:
import numpy as np

def get_features_and_labels(dataset):

  allfeatures = []
  alllabels = []

  for images, labels in dataset:

    processed_image = keras.applications.vgg16.preprocess_input(images)

    features = base_model.predict(processed_image)

    allfeatures.append(features)
    alllabels.append(labels)

  return np.concatenate(allfeatures), np.concatenate(alllabels)

In [ ]:
train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

In [ ]:
train_features.shape

In [ ]:
input = keras.Input(shape=(5,5,512))

x = layers.Flatten()(input)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(input, output)

In [ ]:
model.compile(optimizer = "rmsprop", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
mycallback = [
    keras.callbacks.ModelCheckpoint(
        filepath = "feature_extraction.keras",
        save_best_only=True,
        monitor='val_loss'
    )
]

In [ ]:
history = model.fit(train_features, train_labels, epochs=20, validation_data=(val_features, val_labels), callbacks=mycallback)

## Plot results

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, 21)

plt.plot(epochs, history.history['accuracy'], label="Training Accuracy")
plt.plot(epochs, history.history['val_accuracy'], label="Validation Accuracy")
plt.legend()

plt.figure()
plt.plot(epochs, history.history['loss'], label="Training Loss")
plt.plot(epochs, history.history['val_loss'], label="Validation Loss")
plt.legend()

plt.show()

In [ ]:
#Feature extraction, Data Augumentation and training....

In [ ]:
new_model_base = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False)

len(new_model_base.trainable_weights)

In [ ]:
new_model_base.trainable = False

In [ ]:
len(new_model_base.trainable_weights)

In [ ]:
data_augumentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [ ]:
inputs = keras.Input(shape=(180,180,3))

x = data_augumentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = new_model_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")

model = keras.Model(inputs, outputs)

model.compile (optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy'])


In [ ]:
mycb = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_data_augumentation.keras",
        save_best_only=True,
        monitor='val_loss'
    )
    ]

In [ ]:
dahistory = model.fit(train_dataset, epochs=50, validation_dataset=validation_dataset, callbacks=mycb)

In [ ]:
epochs = range(1, 51)

plt.plot(epochs, dahistory.history['accuracy'], label="Training Accuracy")
plt.plot(epochs, dahistory.history['val_accuracy'], label="Validation Accuracy")
plt.legend()

plt.figure()
plt.plot(epochs, dahistory.history['loss'], label="Training Loss")
plt.plot(epochs, dahistory.history['val_loss'], label="Validation Loss")
plt.legend()

plt.show()

In [ ]:
test_model = keras.models.load_model("feature_extraction_data_augumentation.keras")

test_loss, test_accuracy = test_model.evaluate(test_dataset)

print (f"Test accuracy: {test_accuracy: .3f}")

# Fine tuning model by freezing all layers other than the last 4

In [ ]:
new_model_base.trainable = True

for layer in new_model_base.layers[:-4]:
  layer.trainable = False

In [ ]:
model.compile (optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
mycb2 = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_data_augumentation.keras",
        save_best_only=True,
        monitor='val_loss'
    )
    ]

In [ ]:
dahistory2 = model.fit(train_dataset, epochs=30, validation_dataset=validation_dataset, callbacks=mycb)

In [ ]:
epochs = range(1, 31)

plt.plot(epochs, dahistory2.history['accuracy'], label="Training Accuracy")
plt.plot(epochs, dahistory2.history['val_accuracy'], label="Validation Accuracy")
plt.legend()

plt.figure()
plt.plot(epochs, dahistory2.history['loss'], label="Training Loss")
plt.plot(epochs, dahistory2.history['val_loss'], label="Validation Loss")
plt.legend()

plt.show()